<h1 style = "text-align: center; ">Descriptive Title</h1>
<h2 style = "text-align: center; ">ST445 - Managing and Visualizing Data</h2>
<h3 style = "text-align: center; ">Candidate IDs: 38682, 50450, YYYYY</h3>


### I. Notebook preparation (maybe this section is not needed)

Perhaps we include something similar to this example from "Example 2"

[[Before running this notebook, please make sure you have all necessary modules installed in your environment. Potentially less common modules used include:

google.cloud
dotenv
networkx
geopandas
praw
transformers
plotly.graph_objects
ipywidgets
folium
As usual, they can be installed by running the command pip install [module] in the terminal.

Furthermore, please make sure your Python version is compatible with all the modules. While writing this, it became apparent there might be some compatibility issues with newer Python versions (especially 3.11 and newer). In case you run into any issues, it might be worth trying to run the code with an older version such as Python 3.9.]]

Our complete GitHub repository can be found at the following location: https://github.com/lse-st445/2024-project-data-knows-ball [[Should we put this in the title of our paper??]]

In [1]:
# Import relevant packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import numpy as np

# Install lxml with conda install anaconda::lxml to use HMTL and XML with Python
# conda install openpyxl


### II. Introduction and data description

[[Describe our data sets and pose our research question]]

[[Maybe include data dictionaries of some sort similar to Table 1.3.1 and Table 1.3.2 in "Example 2"]]

### III. Data acquisition

#### III.i. Marketcheck UK API

In [15]:
#Import necessary libraries
!pip install splinter
!pip install selenium splinter
!pip install webdriver-manager

from splinter import Browser
from bs4 import BeautifulSoup as soup
import re
import matplotlib.pyplot as plt
import time
from webdriver_manager.chrome import ChromeDriverManager

# Set up ChromeDriver path automatically
executable_path = {'executable_path': ChromeDriverManager().install()}
#browser = Browser('chrome', **executable_path)
print(executable_path)

# Test the setup
try:
    browser.visit("https://google.com")
    print(f"Successfully opened: {browser.title}")
finally:
    browser.quit()  # Close the browser after testing

# set up splinter
browser = Browser("chrome")

# set up base url
base_url = "https://www.facebook.com/marketplace/london/vehicles?"

# set up search parameters
make = 523665818157652 #stands for Volkswagen
model = 997718013951124 #stands for Golf
min_mileage = 0
max_mileage = 1000000
min_year = 2000
max_year = 2025
min_price = 0
max_price = 1000000
transmission_type = "automatic"

url = f"minPrice={min_price}&maxPrice={max_price}&maxMileage={max_mileage}&maxYear={max_year}&minMileage={min_mileage}&minYear={min_year}&make={make}&model={model}&transmissionType={transmission_type}&exact=false"


{'executable_path': '/Users/imarcolic/.wdm/drivers/chromedriver/mac64/131.0.6778.264/chromedriver-mac-arm64/chromedriver'}


NameError: name 'browser' is not defined

{"message": "Invalid authentication credentials"}


#### III.ii. Webscrapping UK Office of National Statistics (ONS)

##### III.ii.a Unemployment rate and CPIH (time series economic data)

In [3]:
# Write function for webscrapping data from the UK Office of National Statistics
def webscrape_ONS(url):
    '''
    This function webscrapes various tables from the UK ONS and seperates the data 
    into distinct dataframes based on the given periodicity: year, quarter, or month.
    ----------
    Args:
        url: The UK Office of National Statistics url from which to webscrabe the table
    ----------
    Returns:        
        ons_year_df: Dataframe of UK ONS data at the yearly level
        ons_quarter_df: Dataframe of UK ONS data at the quarterly level
        ons_month_df: Dataframe of UK ONS data at the monthly level
    '''
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")

    # Save the table headers to later set as column names for the dataframes
    table_headers = soup.find_all("th")
    table_headers = table_headers[0:2] # We only need the first two columns of data from the ONS
    table_headers = [t.text for t in table_headers]

    ons_data = []

    # Identify and append all webscrapped rows of the ONS table into a dataframe
    for i, row in enumerate(soup.find_all("tr")[2:]): # The frist two rows of ONS tables are headers
        try:
            period, value = row.find_all("td")[0:2] # We only need the first two columns of data from the ONS
            ons_data.append([period.text, value.text])
        except:
            print("Error parsing row #{}".format(i))

    ons_df = pd.DataFrame(ons_data, columns = table_headers)

    # Make the "Value" column data type float instead of string as it was webscrapped
    ons_df = ons_df.astype({"Value": float})

    # Split the data into separate dataframes based on periodicity (year/quarter/month)
    ons_year_df = ons_df[ons_df["Period"].str.len() == 4].reset_index(drop = True) # Year periods will have 4 characters (e.g., "2020")
    ons_quarter_df = ons_df[ons_df["Period"].str.len() == 7].reset_index(drop = True) # Quarter periods will have 7 characters (e.g., "2020 Q1")
    ons_month_df = ons_df[ons_df["Period"].str.len() == 8].reset_index(drop = True) # Month periods will have 8 characters (e.g., "2020 JAN")

    # For dataframes at the yearly level, make year an int type instead of string as it was webscrapped
    ons_year_df = ons_year_df.astype({"Period": int})
    
    # Ensure that all rows present in the original ONS table are present in the three dataframes split based on periodicity
    split_df_len = sum([len(ons_year_df), len(ons_quarter_df), len(ons_month_df)])
    orig_df_len = len(ons_data)
    assert split_df_len == orig_df_len, "ERROR: Not all rows from original ONS table present in corresponding year/quarter/month dataframes"

    return ons_year_df, ons_quarter_df, ons_month_df


In [4]:
# Webscrape UK unemployment and CPIH data tables from the ONS
url_uk_unemp = "https://www.ons.gov.uk/employmentandlabourmarket/peoplenotinwork/unemployment/timeseries/mgsx/lms"
url_uk_cpih = "https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/l55o/mm23"

uk_unemp_year_df, uk_unemp_quarter_df, uk_unemp_month_df = webscrape_ONS(url_uk_unemp)
uk_cpih_year_df, uk_cpih_quarter_df, uk_cpih_month_df = webscrape_ONS(url_uk_cpih)


##### III.ii.b Gross Disposable Household Income (geographic economic data)

In [5]:
# Webscrape data from the UK Office of National Statistics -- Gross Disposable Household Income (GDHI)
url_uk_gdhi = "https://www.ons.gov.uk/economy/regionalaccounts/grossdisposablehouseholdincome/bulletins/regionalgrossdisposablehouseholdincomegdhi/1997to2022"

page = requests.get(url_uk_gdhi)
soup = BeautifulSoup(page.content, "lxml")

# Save the table headers to later set as column names for the dataframes
table_headers = soup.find_all("th")
table_headers = [t.text for t in table_headers]
uk_countries_regions_df = pd.DataFrame(table_headers[8:22]) # The data of the 1st column ("Counties and regions of the UK") is defined as <th> as opposed to <td> and will be combined with rest of data later
table_headers = table_headers[0:5] # We only need the first five columns of data from the ONS


In [6]:
gdhi_data = []

# Identify and append all webscrapped rows of the ONS table into a dataframe
# The ONS table of interest is: Table 1: Gross disposable household income by UK and constituent countries and regions, UK, 2022
# NOTE: 2022 is the most recent year available for ONS data on this topic, this statistical bulletin was released on September 4, 2024
for i, row in enumerate(soup.find_all("tr")[1:15]): # The first row of the ONS tables is headers; Table 1 contains 14 rows
    try:
        pop, gdhi, gdhi_growth, gdhi_index = row.find_all("td")[0:4] # We only need the first four columns of data from the ONS as the 1st column ("Counties and regions of the UK") is defined as <th> as opposed to <td> and will be combined later
        gdhi_data.append([pop.text, gdhi.text, gdhi_growth.text, gdhi_index.text])
    except:
        print("Error parsing row #{}".format(i))

partial_df = pd.DataFrame(gdhi_data)


In [7]:
# Combine the UK countries and regions with the rest of the GDHI data
uk_gdhi_df = pd.concat([uk_countries_regions_df, partial_df], axis = 1)
uk_gdhi_df.columns = table_headers

# Clean the UK GDHI data
uk_gdhi_df.rename({"Countriesandregions of the UK": "Countries and regions of the UK",
                   "Population(million)": "Population (million)",
                   "GDHI perhead (£)": "GDHI per head (£)"},
                  axis = "columns",
                  inplace = True)
uk_gdhi_df["GDHI per head (£)"] = uk_gdhi_df["GDHI per head (£)"].str.replace(",", "")
uk_gdhi_df = uk_gdhi_df.astype({"Population (million)": float, "GDHI per head (£)": int, "Growth in GDHI per head (%)": float, "GDHI per head index (UK=100)": float})
uk_gdhi_df.replace("NorthernIreland", "Northern Ireland", inplace = True)


#### III.iii. Importing UK Office of National Statistics XLSX

##### III.iii.a Median gross weekly earnings (geographic economic data)

In [8]:
# Import XLSX from the UK Office of National Statistics -- Figure 6: Median gross weekly earnings for full-time employees for all local authorities by place of work 
# NOTE: This XLSX is provided via download from the following ONS statistical bulletin, "Employee earnings in the UK: 2024"
    # https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/bulletins/annualsurveyofhoursandearnings/2024
ons_weekly_earnings_df = pd.read_excel("ONS_figure6.xlsx", skiprows = 8)


In [9]:
# Clean the UK weekly earnings data
earnings = ons_weekly_earnings_df["Earnings"]
earnings_numeric = pd.to_numeric(earnings, errors = "coerce")
ons_without_earnings = ons_weekly_earnings_df.drop(columns = "Earnings")

uk_weekly_earnings_df = pd.concat([ons_without_earnings, earnings_numeric], axis = 1)
uk_weekly_earnings_df.dropna(inplace = True, ignore_index = True)
uk_weekly_earnings_df["Local authority name"] = uk_weekly_earnings_df["Local authority name"].str.strip()


#### III.iv. US Bureau of Labor Statistics (BLS) API

In [10]:
# Call the BLS API for the following two datasets between the years of 2019-2024
    # LNS14000000: The USA U-3 unemployment rate ("official rate"); civilian unemployment rate for 16 years and older, seasonally adjusted
        # https://data.bls.gov/timeseries/LNS14000000
    # CUUR0000SA0: CPI - All items in U.S. city average, all urban consumers, not seasonally adjusted
        # https://data.bls.gov/timeseries/CUUR0000SA0
# We are interested in retaining the last 5 years (2020-2024) of data. The reason for including 2019, is so that we can compute the YoY CPI percentage change
# NOTE: because we have not registered we can only querry this API 25 times per day
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['LNS14000000', 'CUUR0000SA0'], "startyear": "2019", "endyear": "2024"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data = data, headers = headers)
json_data = json.loads(p.text)


In [11]:
# Save the BLS unemployment and CPI data within a dataframe
bls_data = []

# NOTE: this code is heavily derived from the BLS API Version 1.0 Python Sample Code
    # https://www.bls.gov/developers/api_python.htm
for series in json_data['Results']['series']:
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']

        if 'M01' <= period <= 'M12':
            bls_data.append([seriesId,year,period,value])

bls_df = pd.DataFrame(bls_data, columns = ["SeriesID", "Year", "Period", "Value"])


In [12]:
# Separate BLS table into unemployment and CPI dataframes
unemp_df = bls_df[bls_df["SeriesID"] == "LNS14000000"].reset_index(drop = True)
cpi_df = bls_df[bls_df["SeriesID"] == "CUUR0000SA0"].reset_index(drop = True)

# Ensure that all rows present in the original BLS table are present in the two split unemployment and CPI dataframes
split_df_len = sum([len(unemp_df), len(cpi_df)])
orig_df_len = len(bls_df)
assert split_df_len == orig_df_len, "ERROR: Not all rows from original BLS table present in corresponding unemployment and CPI dataframes"

# Clean the BLS unemployment data
unemp_df.drop("SeriesID", axis = 1, inplace = True)
unemp_df = unemp_df.astype({"Year": int, "Value": float})

# Clean the BLS CPI data
cpi_df.drop("SeriesID", axis = 1, inplace = True)
cpi_df = cpi_df.astype({"Year": int, "Value": float})


In [13]:
# We are only interested in the previous 5 years of US BLS unemployment data (this is at the monthly level)
us_unemp_month_df = unemp_df[unemp_df["Year"] > 2019].reset_index(drop = True)


In [14]:
# Restructure BLS CPI data to compute year-over-year values
cpi_df_wide = cpi_df.pivot(index = "Year", columns = "Period", values = "Value").pct_change(fill_method = None).reset_index()
cpi_df_long = pd.melt(cpi_df_wide, id_vars = ["Year"], value_vars = ["M01", "M02", "M03", "M04", "M05", "M06", "M07", "M08", "M09", "M10", "M11", "M12"])

# We are only interested in the previous 5 years of US BLS CPI data (this is at the monthly level)
us_cpi_month_df = cpi_df_long.dropna().reset_index(drop = True)
us_cpi_month_df["Value"] = us_cpi_month_df["value"] * 100
us_cpi_month_df.drop("value", axis = 1, inplace = True)
us_cpi_month_df = us_cpi_month_df.sort_values(["Year", "Period"], ascending = False).reset_index(drop = True)


### IV. Data preparation

#### Current datasets that we have:

UK national unemployment data at the year (1971-2023), quarter (1971Q1-2023Q3), and month (1971FEB-2024SEP) level. <br>
Webscrapped from UK Office of National Statistics: https://www.ons.gov.uk/employmentandlabourmarket/peoplenotinwork/unemployment/timeseries/mgsx/lms <br>
**uk_unemp_year_df**, **uk_unemp_quarter_df**, **uk_unemp_month_df**

UK national CPIH data at the year (1989-2024), quarter (1989Q1-2024Q4), and month (1989JAN-2024DEC) level. <br>
Webscrapped from UK Office of National Statistics: https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/l55o/mm23 <br>
**uk_cpih_year_df**, **uk_cpih_quarter_df**, **uk_cpih_month_df**

UK and constituent countries and regions 2022 Gross Disposable Household Income (GDHI) by country/region. <br>
Webscrapped data from the UK Office of National Statistics: https://www.ons.gov.uk/economy/regionalaccounts/grossdisposablehouseholdincome/bulletins/regionalgrossdisposablehouseholdincomegdhi/1997to2022 <br>
**uk_gdhi_df**

Great Britain, April 2024, median gross weekly earnings for full-time employees for all local authorities by place of work. <br>
Import XLSX associated with "Figure 6: Median gross weekly earnings for full-time employees for all local authorities by place of work" provided for download by the UK Office of National Statistics: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/bulletins/annualsurveyofhoursandearnings/2024 <br>
**uk_weekly_earnings_df**

USA monthly (January 2020 - December 2024) U-3 unemployment rate ("official rate"); civilian unemployment rate for 16 years and older, seasonally adjusted. <br>
Called from the US Bureau of Labor Statistics (BLS) API for the following timeseries (LNS14000000):
https://data.bls.gov/timeseries/LNS14000000 <br>
**us_unemp_month_df**

USA monthly (January 2020 - December 2024) year-over-year percentage change for CPI - All items in U.S. city average, all urban consumers, not seasonally adjusted. <br>
Called from the US Bureau of Labor Statistics (BLS) API for the following timeseries (CUUR0000SA0):
https://data.bls.gov/timeseries/CUUR0000SA0 <br>
**us_cpi_month_df**


In [15]:
# Clean and merge datasets



### V. Visualizations

[[Description of what visualizations we decided to include and why]]

In [16]:
# Code for visualizations



[[Explanation/interpretation of the visualizations are depicting]]

### VI. Data modeling

### VII. Conclusion